In [1]:
'''
STEP1
Pre-processing data:
- Defining the data augmentation settings for the training dataset and the validation dataset
- Splitting the dataset w/ a 80:20 ratio
- Things that can be changed: Input image size, normalization parameters, split ratio and batch size

'''
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

img_size = 224
batch_size = 64

data_dir = 'archive/fruits-360_dataset/fruits-360/Training'

train_transforms = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Resize((img_size, img_size)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomCrop((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
valid_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

dataset = ImageFolder(data_dir, transform=train_transforms)
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [2]:
# import matplotlib.pyplot as plt
# import numpy as np
# from PIL import Image


# img_path = 'archive/fruits-360_dataset/fruits-360/Training/Apple Braeburn/0_100.jpg'

# img = Image.open(img_path)

# img_transformed = valid_transforms(img)
# plt.imshow(np.transpose(img_transformed.numpy(), (1, 2, 0)))
# plt.savefig('output.png')

In [3]:
'''
STEP2:
- Building a base model using the pre-trained VGG16 model
- Adding a trainable linear layer on top of the pre-trained model to perform the disease classification task.
'''
import torch.nn as nn
import sklearn
import torchvision.models as models
from torchvision.models.vgg import VGG16_Weights
from sklearn.metrics import accuracy_score

'''
Creates an instance of the pre-trained VGG16 model using the models.vgg16() function from the torchvision library. 
The pretrained=True argument loads the weights for the pre-trained model.
'''

vgg16 = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

'''
Freezes the pre-trained layers so that their weights do not get updated during training.
'''
for param in vgg16.parameters():
    param.requires_grad = False

num_classes = len(train_dataset.dataset.classes)
#sets the num_features to the number of input features for the last layer of the pre-trained VGG16 model's classifier.
num_features = vgg16.classifier[-1].in_features
#num_features = vgg16.fc.in_features

# replaces the last layer of the pre-trained VGG16 model's classifier with a new linear layer
vgg16.classifier[-1] = nn.Linear(num_features, num_classes)
#vgg16.fc = nn.Linear(num_features, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.classifier.parameters(), lr=0.001)
metric = sklearn.metrics.accuracy_score

In [4]:
#print(vgg16)

In [5]:
'''
STEP3
- Training the model
'''
num_epochs = 15
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

vgg16.to(device)

train_losses = []
valid_losses = []
train_accs = []
valid_accs = []

for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0
    valid_loss = 0.0
    valid_acc = 0.0
    
    vgg16.train()
    print("Epoch number: ", epoch)
    for i, (images, labels) in tqdm(enumerate(train_loader)):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() 
        _, preds = torch.max(outputs, 1)
        train_acc += torch.sum(preds == labels.data)

    with torch.no_grad():
        vgg16.eval()
        
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = vgg16(images)
            loss = criterion(outputs, labels)
    
            valid_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            valid_acc += torch.sum(preds == labels.data)
    
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader.dataset)
    valid_loss = valid_loss / len(valid_loader.dataset)
    valid_acc = valid_acc / len(valid_loader.dataset)
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    train_accs.append(train_acc)
    valid_accs.append(valid_acc)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}')

Device: cuda:0
Epoch number:  0


847it [05:20,  2.65it/s]


Epoch [1/15], Train Loss: 0.0233, Train Acc: 0.6014, Valid Loss: 0.7502, Valid Acc: 0.7860
Epoch number:  1


847it [05:16,  2.67it/s]


Epoch [2/15], Train Loss: 0.0156, Train Acc: 0.7050, Valid Loss: 0.5330, Valid Acc: 0.8479
Epoch number:  2


...

Epoch [13/15], Train Loss: 0.0135, Train Acc: 0.7663, Valid Loss: 0.3085, Valid Acc: 0.8999
Epoch number:  13


847it [05:16,  2.68it/s]


Epoch [14/15], Train Loss: 0.0138, Train Acc: 0.7641, Valid Loss: 0.3157, Valid Acc: 0.8956
Epoch number:  14


847it [05:15,  2.68it/s]


Epoch [15/15], Train Loss: 0.0137, Train Acc: 0.7689, Valid Loss: 0.3558, Valid Acc: 0.8858


In [6]:
'''
STEP4
- Plotting the learning curve
'''
# train_losses = [t.item() for t in train_losses]
# valid_losses = [t.item() for t in valid_losses]
# train_accs = [t.item() for t in train_accs]
# valid_accs = [t.item() for t in valid_accs]

# import matplotlib.pyplot as plt

# plt.figure(figsize=(8, 6))
# plt.plot(train_losses, label='Training Loss')
# plt.plot(valid_losses, label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Training and Validation Losses')
# plt.legend()
# plt.show()

# plt.figure(figsize=(8, 6))
# plt.plot(train_accs, label='Training Accuracy')
# plt.plot(valid_accs, label='Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.title('Training and Validation Accuracies')
# plt.legend()
# plt.show()

'\nSTEP4\n- Plotting the learning curve\n'